In [ ]:
# Check Setup: Wo bin ich?
import sys
sys.executable

# Internals: Ein Blick unter die Motorhaube

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification

processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = AutoModelForZeroShotImageClassification.from_pretrained("openai/clip-vit-base-patch32")

## Tokenizer

In [ ]:
processor.tokenizer

### Tokens

In [ ]:
text = "How are you doin'?"

In [ ]:
processor.tokenizer(text)

In [ ]:
for ID in processor.tokenizer(text).input_ids:
    token = processor.tokenizer.convert_ids_to_tokens(ID)
    print(ID, "\t", token)

### Vokabular

In [ ]:
processor.tokenizer.vocab

## Bildprozessor

In [ ]:
processor.image_processor

In [ ]:
from PIL import Image
import requests

url = "https://asset.museum-digital.org/berlin/images/29/202202/103410/auf-der-terrasse-der-milch-mokka-eis-bar-in-der-karl-marx-allee-april-1967-sw-foto-kurt-sc.jpg"

img = Image.open(requests.get(url, stream=True).raw)

In [ ]:
img

In [ ]:
import numpy as np
np.array(img).shape

In [ ]:
processor.image_processor(img)

In [ ]:
processor.image_processor(img).pixel_values[0].shape

In [ ]:
from matplotlib import pyplot as plt
processedImg = processor.image_processor(img).pixel_values[0]
print(processedImg.shape)
processedImg = processedImg.transpose((1,2,0)) 
print(processedImg.shape)
plt.imshow(processedImg)

## Modelle

In [ ]:
model

In [ ]:
image_inputs = processor(images=img, return_tensors="pt")
image_features = model.get_image_features(**image_inputs)

In [ ]:
# Process the text separately
text_inputs = processor(text=["a photo of people sitting in a café", "a photo of a university"], return_tensors="pt", padding=True)
text_features = model.get_text_features(**text_inputs)

In [ ]:
image_inputs

In [ ]:
# Compute cosine similarity between image and text features
similarity = torch.matmul(text_features, image_features.T)
print("Similarity:", similarity)